In [28]:
import numpy as np
import pandas as pd
import io
import glob
import os

In [2]:
def df_from_csv(my_csv):
    return pd.read_csv(io.StringIO('\n'.join(my_csv[2:-1])))

In [110]:
def get_nsys_reports(parent_directory, trial_name, folder_destination='artifacts/'):
    report_paths = glob.glob(parent_directory + '/*-rep')
    time_holder, size_holder = [], []
    for path in report_paths:
        time_csv = !nsys stats --report=gpumemtimesum --format csv -q $path
        size_csv = !nsys stats --report=gpumemsizesum --format csv -q $path
        time_holder.append(df_from_csv(time_csv))
        size_holder.append(df_from_csv(size_csv))
    time_df = pd.concat(time_holder, axis=0, ignore_index=True)
    size_df = pd.concat(size_holder, axis=0, ignore_index=True)
    time_df.to_csv(folder_destination + trial_name +'-time.csv', index=False)
    size_df.to_csv(folder_destination + trial_name +'-size.csv', index=False)
    return time_df, size_df

In [176]:
trial_names = ['r50', 'r50-256bs', 'r50-phylum', 'r50-pytorch', 'r50-1gpu', 
               'r50-accum-1gpu', 'r50-accum-4gpu', 'r50-tl', 'r101']

In [159]:
folder_path = '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3443950'
file_name = os.listdir(folder_path)[0]
file_path = folder_path + f'/{file_name}' 
file_name

'nsys_report.nid001544.129059.nsys-rep'

In [100]:
#leads to unyieldy output without grep, couldn't figure out how to only pull int starting lines
full_file = !nsys stats --report=gpukernsum --format csv -q $file_path
#len(full_file)#full_file

nccl = !nsys stats --report=gpukernsum --format csv -q $file_path | grep nccl
#nccl #if want to see nccl activities

#total time is second column - first col is % time
#nccl[0].split(',')[1]

#this should get the total number of seconds for nccl related activities
np.array([x.split(',')[1] for x in nccl], dtype=float).sum() / 1e9

59.748935084

In [228]:
def get_nccl_sum(file_path):
    nccl_activities = !nsys stats --report=gpukernsum --format csv -q {file_path} | grep nccl
    total = np.array([x.split(',')[1] for x in nccl_activities], dtype=float).sum() / 1e9
    return total

#hardcoded in 150 of the activities -- played around and not much higher afte 
def get_total_kern(file_path):
    tot_activities = !nsys stats --report=gpukernsum --format csv -q {file_path}
    tot_activities = tot_activities[3:10]
    total = np.array([x.split(',')[1] for x in tot_activities], dtype=float).sum() / 1e9
    return total

In [229]:
def get_report_paths(parent_directory):
    report_paths = glob.glob(parent_directory + '/*-rep')
    return report_paths

In [230]:
get_total_kern(file_path), get_nccl_sum(file_path)

(70.795084467, 59.748935084)

In [231]:
def get_kern_time(parent_dir):
    report_paths = get_report_paths(parent_dir)
    file_path = report_paths[0]
    nccl_time = get_nccl_sum(file_path)
    total_time = get_total_kern(file_path)
    return total_time, nccl_time

In [232]:
t_names = trial_names[:2]
folder_one = '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3443950'
folder_two = '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3444217'
trial_tuple = tuple(zip(t_names, [folder_one, folder_two]))

In [233]:
get_kern_time(folder_one)

(70.795084467, 59.748935084)

In [234]:
def get_kern_dfs(trial_tuple):
    tot_times = []
    nccl_times = []
    trial_names = []
    for trial_name, path in trial_tuple:
        total_time, nccl_time = get_kern_time(path)
        trial_names.append(trial_name)
        nccl_times.append(nccl_time)
        tot_times.append(total_time)
    df = pd.DataFrame(zip(tot_times, nccl_times), columns=['total_time', 'nccl_time'], index=trial_names)
    return df

In [235]:
get_kern_dfs(trial_tuple)

,total_time,nccl_time
r50,70.795084,59.748935
r50-256bs,86.054444,75.542611


In [236]:
trial_names = ['r50', 'r50-256bs', 'r50-phylum', 'r50-pytorch',
               #'r50-1gpu', 
               #'r50-accum-1gpu', 
               'r50-accum-4gpu', 'r50-tl', 'r101']

In [237]:
paths=  ['/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3443950',
        '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3444217',
        '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3443958',
        '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3443968',
        #'/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3444142',
        #'/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3444149',
        '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3444641',
        '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet50/C/test00/train.3444386',
        '/global/homes/a/azaidi/dt_cfs/azaidi/new_trials/speed_trials/train/datasets/nsys_new/chunks_W1024_S128/resnet101/C/test00/train.3444019']

In [238]:
len(paths)

7

In [239]:
[len(get_report_paths(p)) for p in paths]

[4, 4, 4, 4, 4, 4, 4]

In [240]:
trial_tup = tuple(zip(trial_names, paths))

In [241]:
get_kern_dfs(trial_tup)

,total_time,nccl_time
r50,70.795084,59.748935
r50-256bs,86.054444,75.542611
r50-phylum,71.562527,60.404087
r50-pytorch,98.399284,86.476833
r50-accum-4gpu,12.693271,0.589333
r50-tl,126.135461,12.202738
r101,17.617514,2.164545
